### 1. Setup & CSV einlesen


In [1]:
# -----------------------------------------------------------
# 1) Import necessary libraries
# -----------------------------------------------------------
import pandas as pd
import numpy as np
import re
import sys
!{sys.executable} -m pip install sqlalchemy pymysql cryptography

# Import utility functions
from db_utils import setup_display_settings, write_to_db

# Setup display settings
setup_display_settings()

# -----------------------------------------------------------
# 2) Load the dataset (adjust filename if needed)
# -----------------------------------------------------------
df = pd.read_csv("schweizer_wanderwege_alle.csv")

# First quick look
df.head()

Defaulting to user installation because normal site-packages is not writeable
✅ Display settings configured


,web_scraper_order,web_scraper_start_url,detail_url,title,location,difficulty_level,duration,distance,physical_demand,ascent,descent,target_group,season,recommended_period
0,1766396370-1,https://www.schweizer-wanderwege.ch/de/wanderv...,https://www.schweizer-wanderwege.ch/de/wanderv...,Husky-Traum,Muotathal,NaN,NaN,4m,hoch,NaN,NaN,"Zyklus 1 (Grundschule / 6 - 9 Jahre alt), Zykl...",ganzjährig,NaN
1,1766396372-2,https://www.schweizer-wanderwege.ch/de/wanderv...,https://www.schweizer-wanderwege.ch/de/wanderv...,Aufregende Ausblicke auf dem Hochalp Trail,"Urnäsch, Anker\n ...",NaN,6h 45min,11.4km,hoch,720m,720m,NaN,Winter,Dezember - März
2,1766396375-3,https://www.schweizer-wanderwege.ch/de/wanderv...,https://www.schweizer-wanderwege.ch/de/wanderv...,Den ganzen Tag im Schnee,Atzmännig SSA\n ...,NaN,1h 40min,4.2km,mittel,140m,510m,für Familien,Winter,Dezember - März
3,1766396377-4,https://www.schweizer-wanderwege.ch/de/wanderv...,https://www.schweizer-wanderwege.ch/de/wanderv...,Panorama Trail in Obwalden,Langis\n ...,NaN,4h 30min,12.3km,hoch,565m,565m,NaN,Winter,Dezember - März
4,1766396379-5,https://www.schweizer-wanderwege.ch/de/wanderv...,https://www.schweizer-wanderwege.ch/de/wanderv...,Schneeschuhwanderung auf den Chasseron,"Les Rasses, village\n ...",NaN,4h 20min,10km,hoch,450m,450m,NaN,Winter,Dezember - März


### 2. Remove unnecessary scraper columns

In [2]:
# -----------------------------------------------------------
# Remove columns created automatically by the web-scraper
# These do not contain useful analytical information
# -----------------------------------------------------------
columns_to_drop = ["web_scraper_order", "web_scraper_start_url"]

df = df.drop(columns=columns_to_drop, errors="ignore")

# Rename the "detail_url" column to a simpler name
df = df.rename(columns={"detail_url": "url"})

df.head()


,url,title,location,difficulty_level,duration,distance,physical_demand,ascent,descent,target_group,season,recommended_period
0,https://www.schweizer-wanderwege.ch/de/wanderv...,Husky-Traum,Muotathal,NaN,NaN,4m,hoch,NaN,NaN,"Zyklus 1 (Grundschule / 6 - 9 Jahre alt), Zykl...",ganzjährig,NaN
1,https://www.schweizer-wanderwege.ch/de/wanderv...,Aufregende Ausblicke auf dem Hochalp Trail,"Urnäsch, Anker\n ...",NaN,6h 45min,11.4km,hoch,720m,720m,NaN,Winter,Dezember - März
2,https://www.schweizer-wanderwege.ch/de/wanderv...,Den ganzen Tag im Schnee,Atzmännig SSA\n ...,NaN,1h 40min,4.2km,mittel,140m,510m,für Familien,Winter,Dezember - März
3,https://www.schweizer-wanderwege.ch/de/wanderv...,Panorama Trail in Obwalden,Langis\n ...,NaN,4h 30min,12.3km,hoch,565m,565m,NaN,Winter,Dezember - März
4,https://www.schweizer-wanderwege.ch/de/wanderv...,Schneeschuhwanderung auf den Chasseron,"Les Rasses, village\n ...",NaN,4h 20min,10km,hoch,450m,450m,NaN,Winter,Dezember - März


### 3. Remove duplicate routes using the unique URL

In [3]:
# -----------------------------------------------------------
# Remove duplicate hiking routes based on the URL.
# The URL uniquely identifies a hiking detail page.
# -----------------------------------------------------------
before = len(df)
df = df.drop_duplicates(subset=["url"])
after = len(df)

print("Rows before removing duplicates:", before)
print("Rows after:", after)


Rows before removing duplicates: 375
Rows after: 375


### 4. Convert distance to a numeric float (km)

In [4]:
# -----------------------------------------------------------
# Convert values like "11km" into the numeric value 11.0
# -----------------------------------------------------------

def parse_distance_km(x):
    """
    Convert strings such as '11km' or '11 km' to a float.
    If the value cannot be parsed, return NaN.
    """
    if pd.isna(x):
        return np.nan

    m = re.match(r"([\d.]+)\s*(km|m)$", str(x).lower())
    if not m:
        return np.nan

    value, unit = m.groups()
    return float(value) / 1000 if unit == "m" else float(value)

df["distance_km"] = df["distance"].apply(parse_distance_km)

df[["distance", "distance_km"]].head()


,distance,distance_km
0,4m,0.00
1,11.4km,11.40
2,4.2km,4.20
3,12.3km,12.30
4,10km,10.00


### 5. Convert duration into total minutes

In [5]:
# -----------------------------------------------------------
# Convert strings like "3h 15min" into total minutes.
# Example: "3h 15min" → 195 minutes
#          "2h" → 120
#          "45min" → 45
# -----------------------------------------------------------

def duration_to_minutes(x):
    if pd.isna(x):
        return np.nan
    
    text = str(x).lower().replace(" ", "")
    
    # Extract hours and minutes
    hours = 0
    minutes = 0

    h_match = re.search(r"(\d+)h", text)
    m_match = re.search(r"(\d+)min", text)

    if h_match:
        hours = int(h_match.group(1))
    if m_match:
        minutes = int(m_match.group(1))

    total = hours * 60 + minutes
    return total if total > 0 else np.nan

df["duration_min"] = df["duration"].apply(duration_to_minutes)

df[["duration", "duration_min"]].head()


,duration,duration_min
0,NaN,NaN
1,6h 45min,405.00
2,1h 40min,100.00
3,4h 30min,270.00
4,4h 20min,260.00


### 6. Convert ascent & descent into integer meters

In [6]:
# -----------------------------------------------------------
# Convert values like "310m" into integer values such as 310
# -----------------------------------------------------------

def height_to_m(x):
    if pd.isna(x):
        return np.nan
    match = re.findall(r"\d+", str(x))
    return int(match[0]) if match else np.nan

df["ascent_m"]  = df["ascent"].apply(height_to_m)
df["descent_m"] = df["descent"].apply(height_to_m)

df[["ascent", "ascent_m", "descent", "descent_m"]].head()


,ascent,ascent_m,descent,descent_m
0,NaN,NaN,NaN,NaN
1,720m,720.00,720m,720.00
2,140m,140.00,510m,510.00
3,565m,565.00,565m,565.00
4,450m,450.00,450m,450.00


### 7. Split the location field into area, region, and canton

In [7]:
# -----------------------------------------------------------
# The 'location' field sometimes looks like:
# "Abländschen, Mittelberg — Saanen • BE"
#
# We split this into:
# - area        (left side of —)
# - region      (right side of —)
# - canton      (after the bullet •)
# -----------------------------------------------------------

def extract_canton(loc):
    if pd.isna(loc): return np.nan
    # Entferne Zeilenumbrüche und überflüssige Whitespaces
    loc_clean = ' '.join(str(loc).split())
    parts = loc_clean.split("•")
    return parts[-1].strip() if len(parts) > 1 else np.nan

def extract_region(loc):
    if pd.isna(loc): return np.nan
    loc_clean = ' '.join(str(loc).split())
    left = loc_clean.split("•")[0]
    if "—" in left:
        return left.split("—")[1].strip()
    return left.strip()

def extract_area(loc):
    if pd.isna(loc): return np.nan
    loc_clean = ' '.join(str(loc).split())
    left = loc_clean.split("•")[0]
    if "—" in left:
        return left.split("—")[0].strip()
    return left.strip()

df["canton"] = df["location"].apply(extract_canton)
df["region"] = df["location"].apply(extract_region)

df["area"]   = df["location"].apply(extract_area)

df[["location", "area", "region", "canton"]].head()

,location,area,region,canton
0,Muotathal,Muotathal,Muotathal,NaN
1,"Urnäsch, Anker\n ...","Urnäsch, Anker","Urnäsch, Anker",AR
2,Atzmännig SSA\n ...,Atzmännig SSA,"Atzmännig, Schutt",SG
3,Langis\n ...,Langis,Langis,OW
4,"Les Rasses, village\n ...","Les Rasses, village","Les Rasses, village",VD


### 8. Encode difficulty & physical demand (categorical → numeric)

In [8]:
# -----------------------------------------------------------
# Convert categorical difficulty levels to numeric values
# Example: T1→1, T2→2, ..., T6→6
# WICHTIG: difficulty_level bleibt mit T-Präfix (z.B. "T2")
# -----------------------------------------------------------

difficulty_map = {
    "T1": 1, "T2": 2, "T3": 3,
    "T4": 4, "T5": 5, "T6": 6
}

def clean_difficulty_level(x):
    """
    Entfernt Whitespaces, normalisiert auf Großbuchstaben und
    gibt None zurück, wenn kein gültiges T1..T6 vorhanden ist.
    """
    if pd.isna(x):
        return None
    s = re.sub(r'\s+', '', str(x))
    if s == "" or s.lower() == "nan":
        return None
    s = s.upper()
    return s if re.match(r"^T[1-6]$", s) else None

# Anwenden: difficulty_level wird jetzt None statt 'nan'/'NaN' als String
df["difficulty_level"] = df["difficulty_level"].apply(clean_difficulty_level)

# difficulty_num als nullable Integer (pd.NA -> SQL NULL)
df["difficulty_num"] = df["difficulty_level"].map(difficulty_map).astype("Int64")


df[["difficulty_level", "difficulty_num"]].head()

,difficulty_level,difficulty_num
0,None,<NA>
1,None,<NA>
2,None,<NA>
3,None,<NA>
4,None,<NA>


In [9]:
# -----------------------------------------------------------
# Convert "leicht/mittel/hoch" to numeric 1/2/3
# -----------------------------------------------------------

physical_demand_map = {"leicht":1, "mittel":2, "hoch":3}

df["physical_demand"] = df["physical_demand"].astype(str).str.lower().str.strip()
df["physical_demand_num"] = df["physical_demand"].map(physical_demand_map)

df[["physical_demand", "physical_demand_num"]].head()


,physical_demand,physical_demand_num
0,hoch,3.00
1,hoch,3.00
2,mittel,2.00
3,hoch,3.00
4,hoch,3.00


### 9. Open-Data-Enrichment with longitude & latitude

In [10]:
import os
import time
import requests
import pandas as pd
import numpy as np

CACHE_FILE = "geocoding_cache.csv"
RATE_LIMIT_SECONDS = 0.25
REQUEST_TIMEOUT_SECONDS = 2


def geocode_osm(place: str):
    """
    Query OpenStreetMap Nominatim API to get latitude and longitude for a place.
    Always prints the final request URL (or a clear message if the request couldn't be built/sent).
    Returns (lat, lon) as floats, or (None, None) if nothing is found / error.
    """
    if pd.isna(place) or str(place).strip() == "":
        print("API Request: (skipped - empty place)")
        return None, None

    url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": str(place),
        "format": "json",
        "addressdetails": 1,
        "limit": 1,
        "countrycodes": "ch",
    }
    headers = {
        "User-Agent": "SwissHikeGeocoder/1.0 (contact: you@example.com)"
    }

    try:
        # Prepare first so we can ALWAYS print the final URL, even if the request fails.
        req = requests.Request("GET", url, params=params, headers=headers)
        prepped = req.prepare()

        print(f"API Request: {prepped.url}")

        with requests.Session() as s:
            # timeout=2 applies to both connect+read when passed as a single number
            response = s.send(prepped, timeout=REQUEST_TIMEOUT_SECONDS)
            response.raise_for_status()
            data = response.json()

        if not data:
            return None, None

        return float(data[0]["lat"]), float(data[0]["lon"])

    except Exception as e:
        print(f"API Error for place='{place}': {type(e).__name__}: {e}")
        return None, None

    finally:
        time.sleep(RATE_LIMIT_SECONDS)


def ensure_cache_file_exists(cache_file: str = CACHE_FILE):
    if not os.path.exists(cache_file):
        with open(cache_file, "w", encoding="utf-8") as f:
            f.write("url,latitude,longitude\n")


def normalize_cache_file(cache_file: str = CACHE_FILE):
    ensure_cache_file_exists(cache_file)

    cache_df = pd.read_csv(cache_file)

    if cache_df.empty:
        return {}

    cache_df["latitude"] = pd.to_numeric(cache_df["latitude"], errors="coerce")
    cache_df["longitude"] = pd.to_numeric(cache_df["longitude"], errors="coerce")
    cache_df = cache_df.dropna(subset=["url", "latitude", "longitude"])
    cache_df["url"] = cache_df["url"].astype(str)

    cache_df = cache_df.drop_duplicates(subset=["url"], keep="last")
    cache_df.to_csv(cache_file, index=False)

    cache_dict = dict(zip(cache_df["url"], zip(cache_df["latitude"], cache_df["longitude"])))
    print(f"Loaded {len(cache_dict)} cached coordinates (deduplicated)")
    return cache_dict


def get_coordinates(place, identifier, cache_dict):
    if identifier in cache_dict:
        lat, lon = cache_dict[identifier]
        print(f"Found in cache: {place} -> {lat}, {lon}")
        return float(lat), float(lon)

    lat, lon = geocode_osm(place)

    if lat is None or lon is None:
        print(f"Not found: {place}")
        return np.nan, np.nan

    print(f"Geocoded: {place} -> {lat}, {lon}")
    return lat, lon


def append_to_cache(cache_file, identifier, lat, lon):
    if pd.isna(lat) or pd.isna(lon):
        return

    with open(cache_file, "a", encoding="utf-8") as f:
        f.write(f"{identifier},{lat},{lon}\n")
        f.flush()


def geocode_dataframe(df: pd.DataFrame, cache_file: str = CACHE_FILE) -> pd.DataFrame:
    df = df.copy()
    df["latitude"] = np.nan
    df["longitude"] = np.nan

    cache_dict = normalize_cache_file(cache_file)

    for idx, row in df.iterrows():
        place = row.get("area")
        identifier = row.get("url")

        if pd.isna(identifier) or str(identifier).strip() == "":
            lat, lon = geocode_osm(place)
            df.at[idx, "latitude"] = lat if lat is not None else np.nan
            df.at[idx, "longitude"] = lon if lon is not None else np.nan
            continue

        identifier = str(identifier)

        lat, lon = get_coordinates(place, identifier, cache_dict)

        df.at[idx, "latitude"] = lat
        df.at[idx, "longitude"] = lon

        if identifier not in cache_dict and not (pd.isna(lat) or pd.isna(lon)):
            append_to_cache(cache_file, identifier, lat, lon)
            cache_dict[identifier] = (lat, lon)

    normalize_cache_file(cache_file)
    return df


# -----------------------------------------------------------
# Apply geocoding
# -----------------------------------------------------------
df = geocode_dataframe(df)

Loaded 342 cached coordinates (deduplicated)
Found in cache: Muotathal -> 46.974668, 8.7658029
Found in cache: Urnäsch, Anker -> 47.3097591, 9.2762871
API Request: https://nominatim.openstreetmap.org/search?q=Atzm%C3%A4nnig+SSA&format=json&addressdetails=1&limit=1&countrycodes=ch
API Error for place='Atzmännig SSA': HTTPError: 403 Client Error: Forbidden for url: https://nominatim.openstreetmap.org/search?q=Atzm%C3%A4nnig+SSA&format=json&addressdetails=1&limit=1&countrycodes=ch
Not found: Atzmännig SSA
Found in cache: Langis -> 46.8929885, 8.1252606
Found in cache: Les Rasses, village -> 46.8308907, 6.5547359
Found in cache: Veltheim AG, Bären -> 47.4369232, 8.1474092
Found in cache: Bibern SH, Dorf -> 47.7722392, 8.6785329
Found in cache: Spreitenbach -> 47.4182444, 8.3640342
Found in cache: Anzère, télécabine -> 46.2948177, 7.3949904
Found in cache: Les Paccots, Les Rosalys -> 46.5174765, 6.9607181
Found in cache: Zweisimmen, Sparenmoos -> 46.5511623, 7.3396399
API Request: https://n

### 10. Save the cleaned dataset

In [11]:
# -----------------------------------------------------------
# Export the cleaned dataset so it can be used in the next steps
# (EDA, modelling, database storage, etc.)
# -----------------------------------------------------------

# Convert duration_min, ascent_m, and descent_m to integer for cleaner output
# Only convert if the value is not NaN
df["duration_min"] = df["duration_min"].apply(lambda x: int(x) if pd.notna(x) else np.nan)
df["ascent_m"] = df["ascent_m"].apply(lambda x: int(x) if pd.notna(x) else np.nan)
df["descent_m"] = df["descent_m"].apply(lambda x: int(x) if pd.notna(x) else np.nan)

# Bereinige location: entferne Zeilenumbrüche und überflüssige Whitespaces
df["location_clean"] = df["location"].apply(lambda x: ' '.join(str(x).split()) if pd.notna(x) else x)

# Extrahiere Canton aus der bereinigten Location (nach dem • Symbol)
def extract_canton_clean(loc):
    if pd.isna(loc): return np.nan
    if "•" in str(loc):
        return str(loc).split("•")[-1].strip()
    return np.nan

df["canton_clean"] = df["location_clean"].apply(extract_canton_clean)

# Entferne "• Canton" aus location, behalte nur die Ortsnamen
def clean_location_remove_canton(loc):
    if pd.isna(loc): return loc
    if "•" in str(loc):
        return str(loc).split("•")[0].strip()
    return str(loc).strip()

df["location_clean"] = df["location_clean"].apply(clean_location_remove_canton)

# Wähle nur die gewünschten Spalten aus
columns_to_export = [
    "url",
    "title", 
    "location_clean",
    "canton_clean",
    "difficulty_level",
    "difficulty_num",
    "duration_min",
    "distance_km",
    "ascent_m",
    "descent_m",
    "physical_demand",
    "latitude",
    "longitude"
]

df_export = df[columns_to_export]

# Benenne Spalten für sauberes CSV
df_export = df_export.rename(columns={
    "location_clean": "location",
    "canton_clean": "canton"
})

print(f"Prepared {len(df_export)} rows with {len(columns_to_export)} columns for MySQL export")


✅ Prepared 375 rows with 13 columns for MySQL export


### 11. Store data in MySQL database

In [12]:
# -----------------------------------------------------------
# Write cleaned and enriched data to MySQL using utility function
# Make sure Docker container is running: docker compose up -d
# -----------------------------------------------------------

# Nutze write_to_db() Funktion aus db_utils.py
write_to_db(df_export, table_name='wanderwege', if_exists='replace')

# Verify data was written correctly
from db_utils import read_from_db
verification_df = read_from_db("SELECT COUNT(*) as count FROM wanderwege")
print(f"Verification: {verification_df['count'][0]} rows in database")


✅ Successfully stored 375 rows in table 'wanderwege'
✅ Loaded 1 rows, 1 columns from database

✓ Verification: 375 rows in database
